<a href="https://colab.research.google.com/github/fbeilstein/topological_data_analysis/blob/master/lecture_6_persistence_homology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #Different TDA Usage Examples

%%html
<link   rel="stylesheet" href="https://fbeilstein.github.io/machine_learning/js_common/styles.css"/>
<script src="https://fbeilstein.github.io/machine_learning/js_common/script.js"></script>
<script>
load_slides_from(
[
`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Topological features and NNs
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/topological_features_cnn.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Topological analysis of proteins
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/barcode_diagrams_on_proteins.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Persistent Homology to Detect DNA-Recombination
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/finches_recombination_tda.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
<br><br>
<a href="https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005071">
From Pablo G. Cámara, Arnold J. Levine, and Raúl Rabadán, 'Inference of ancestral
recombination graphs through topological data analysis',
PLOS Computational Biology 12.8 (2016). doi: 10.1371/journal.pcbi.1005071
</a>
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Topological analysis of proteins
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/secondary_structure_to_barcode.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Persistent Homology and Machine Learning
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/tda_and_ml_diagram.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Dimensionality Reduction for Transcriptomics
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/transcriptomics_dimensionality_reduction.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="8">
Time Series Analysis
</font>
<img src="https://lewtun.github.io/hepml/images/sliding_window_embedding.png" width="800px" style="float: right; margin: 7px 0 7px 7px;">
</div>
`
  ]
)
</script>

* A number of computational problems in theoretical physics can be reduced to comparing points or clouds of points, be these points measured or calculated positions of real physical objects (e.g. stars, galaxies, gas molecules) or even states of a certain system in its configuration space.
* We are considering the problem of comparing two pointclouds.
* There is no general straightforward method to do that: considering each individual point leads to “informational overflow,” ambiguity, and in most cases will be too sensitive to small changes in the positions of points.
* Consider comparing gas in two baloons. What if you have no thermodynamics?


**Here comes Topology**

* To solve the problem, one can avoid considering individual points, but
extract instead certain generalized information about their distribution that will comprise a manageable amount of numbers and then look for some notion of distance for these sets of numbers.
* $$
\begin{array}{llllllll}
\text{Point Cloud 1} & \rightarrow &\text{Topological Magic} &\rightarrow &\text{Numbers} & \searrow \\
 & & & & & \text{Comparison Procedure} & \rightarrow & \text{Result} \\
\text{Point Cloud 2} & \rightarrow &\text{Topological Magic} &\rightarrow &\text{Numbers} & \nearrow \\
\end{array}
$$


**Topological Magic be like**

$$\require{AMScd}
\begin{CD}
\text{Point Cloud} @>>> \text{Manifold} @>\text{topology}>> \text{Complex (Simplicial, $\Delta$-, CW-)} @>\text{algebra}>> \text{Homology groups} @>>> \text{Betti Numbers}\\
\end{CD}$$

### Simplicial Complex: a Bridge between Topology and Algebra

The first step in the analysis provided is the construction of a special mathematical object--- a simplicial complex--- from the dataset given.
A simplicial complex is so special because it bridges topology and algebra.
Let's first define a simplex as a geometrical object.

**Example of geometric simplices:**

![img](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Simplexes.jpg/640px-Simplexes.jpg)


**Definition.**
A set of points $\{a_0,\dots,a_n\}$ in some Euclidean space $\mathbb{R}^m$ is said to be **independent** if vectors $a_1 - a_0$, $a_2 - a_0$,...,$a_n - a_0$ are linearly independent.


**Theorem.**
Any subset of a linearly independent set of points is linearly independent.

$\blacktriangleleft$ Obvious if $a_0$ is in the subset.
If not take any $a_k$ and consider vectors $a_i - a_k$ together with $a_k - a_0$.
This set is linearly independent due to $(a_i - a_k) + (a_k - a_0) = a_i - a_0$, thus a set of $a_i - a_k$ only is linearly independent as well. $\blacksquare$


**Definition.**
A **geometric $n$-simplex** $\sigma^n$ is a set of points
$$
\sigma^n = \left\{\left.
\sum_{i=0}^n \lambda_i a_i\right|
\lambda_i \geq 0,\quad \sum_{i=0}^n \lambda_i = 1
\right\}
$$
where $a_i$ are independent points in some Euclidean space $\mathbb{R}^m$; $\sigma^n$ is given the subspace topology.
The subspace of $\sigma^n$ of the points with  $\lambda_{k_0}=0,\dots,\lambda_{k_p}=0$ for a set of indices $\{k_0,\dots,k_p\}$ is called a **face of $\sigma^n$**.
The face is **proper** if it's not empty or the whole $\sigma^n$.


**Definition.**
A **geometric simplicial complex $K$** is a finite set of simplices all contained in $\mathbb{R}^m$ and satisfying:
* if $\sigma^n$ is a simplex of $K$ and $\tau^p$ is a face of $\sigma^n$, then $\tau^p$ is in $K$;
* if $\sigma^n$ and $\tau^p$ are simplices in $K$, then $\sigma^n \cap \tau^p$ is either empty or a common face of $\sigma^n$ and $\tau^p$.

The **dimension** of $K$ is the maximal dimension of its simplices.


**Definition.**
A union of all geometric simplices of a geometric simplicial complex $K$ equipped with a subset topology (with respect to $\mathbb{R}^m$) is called a **polyhedron $|K|$**.



In [ ]:
#@title #Simplicial Complex

%%html
<link   rel="stylesheet" href="https://fbeilstein.github.io/machine_learning/js_common/styles.css"/>
<script src="https://fbeilstein.github.io/machine_learning/js_common/script.js"></script>
<script>
load_slides_from(
[
`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="6">
Simplicial Complex
</font>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Simplicial_complex_example.svg/494px-Simplicial_complex_example.svg.png"
width="350px" style="float: left; margin: 7px 0 7px 7px;">
</div>
`,`
<div style="max-width:750px; height:200px;">
<font face="helvetica" color="black" size="6">
Invalid Complex
</font>
<img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/invalid_simplicial_complex.png"
width="350px" style="float: left; margin: 7px 0 7px 7px;">
</div>
`
]
)
</script>

**Sidenote:**  
Loosely speaking Simplicial complexes, $\Delta$-complexes, and CW-complexes are all constructed by gluing together simplices. However, for each one, there are different rules for what kinds of "gluings" you are allowed to use.
* For CW-complexes, you are allowed to use almost any gluing. Specifically, a CW-complex is constructed by induction, where at each step, you adjoin a new simplex by gluing its boundary to the complex you have already by any map. The only restriction to this gluing process is that you have to add simplices in increasing order of dimension.
* $\Delta$-complex is a union of simplices which are glued together by just gluing their faces together in the "obvious" linear way, rather than by arbitrary complicated continuous maps. Note, however, that some faces of a single simplex might get glued to each other: you can start with a single vertex and then add an edge both of whose boundary vertices are the one vertex you started with (this gives you a circle).
* Simplicial complexes are $\Delta$-complexes which satisfy even more restrictions: it is required to map different faces of $\partial \Delta n$  to different ($n-1$)-simplices, so the situation discussed at the end of the previous paragraph cannot happen. In addition, you are not allowed to add two different $n$-simplices with the same set of vertices.

It is often more simple to use CW- or $\Delta$ complexes for theoretical calculations, but simplicial complexes for computer-aided calculations. Guuess, what we prefer :)


But what if we consider a simplicial complex as a purely combinatorial object that only defines relations between its vertices without referring to geometry?

**Definition 1.**
An **abstract simplicial complex $\mathcal{K}$** is a finite set of elements $a_0,\dots,a_N$ called **(abstract) vertices**, together with a collection of subsets of the form $(a_{i_0},\dots,a_{i_n})$ called **(abstract) simplices** closed under the operation of taking a subset (subset of a simplex is itself a simplex).
A simplex $(a_{i_0},\dots,a_{i_n})$ contains $n+1$ points, while $n$ is called the **dimension of the simplex**.
The **dimension of $\mathcal{K}$** is the maximum of the dimensions of its simplices.



**Definition.**
Let $K$ be a geometric simplicial complex and $\mathcal{K}$ be an abstract simplicial complex such that there exists a bijection between their vertices and a subset of vertices being simplex in $\mathcal{K}$ if and only if they correspond to the vertices of some simplex in $K$. $\mathcal{K}$ is called an **abstraction** of $K$ and $K$ is called a **realization** of $\mathcal{K}$.


Obviously, any geometric simplicial complex can be abstracted, but surprisingly we can always do the other way around with **Theorem 1**.
The realization is unique in some sense, thus abstract and geometrical simplices are firmly connected.


**Definition.**
Given geometric simplicial complexes $K$ and $L$, a **simplicial map** is a function $f: K \to L$ with the following properties:
* if $a$ is a vertex of $K$, then $f(a)$ is a vertex of $L$;
* if $a_0$,...,$a_n$ are vertices of a simplex $\sigma^n$ of $K$, then $f(a_0)$,...,$f(a_n)$ span a simplex in $L$ (note: repeats possible) and $f(\sum_{i=0}^n \lambda_i a_i) = \sum_{i=0}^n \lambda_i f(a_i)$ ("linear" on each simplex).



**Theorem 1.**
An $n$-dimensional abstract simplex $\mathcal{K}$ has a realization in $\mathbb{R}^{2n+1}$.
Moreover, let $K_1$ and $K_2$ be realisations of $\mathcal{K}$, then there exists a homeomorphism being a simplicial map $f : |K_1| \to |K_2|$.

$\blacktriangleleft$ see~\cite{maunder} $\blacksquare$


The key point is that the geometric information is retained within the abstraction of a geometric simplicial complex. If any information were lost during the abstraction, we should have been able to construct at least two non-homeomorphic realizations of $\mathcal{K}$ that fill in differently the missing piece of information. However, the **Theorem 1** forbids that, thus abstraction retains all the topological information. This connection paves the way for a transition between geometry and algebra.